In [22]:
import pandas as pd
import numpy as np
import scipy as sp

In [4]:
users = pd.read_csv("data/ml-latest-small/ratings.csv",usecols = ['userId','movieId','rating'])
movies = pd.read_csv("data/ml-latest-small/movies.csv",usecols = ['movieId','title'])

In [5]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [8]:
users = pd.merge(users,movies,on = "movieId")

In [9]:
users.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [12]:
combine_users_ratings = users.dropna(axis = 0,subset= ['title'])

In [13]:
movies_ratingCount = (combine_users_ratings.
groupby('title')['rating'].
count().
reset_index().
rename(columns = {'rating':'total_ratings'})
[['title','total_ratings']]
)
movies_ratingCount.head()

,title,total_ratings
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [14]:
total_rating = pd.merge(users,movies_ratingCount, on = "title", how = "left")

In [17]:
total_rating.head()

,userId,movieId,rating,title,total_ratings
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [18]:
popularity = 50

rating_popular_movie = total_rating.query('total_ratings > @popularity')

In [19]:
rating_popular_movie.head()

,userId,movieId,rating,title,total_ratings
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [20]:
movies_matrix = rating_popular_movie.pivot_table(index = "title", columns = "userId", values = "rating").fillna(0)

In [21]:
movies_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [24]:
from scipy.sparse import csr_matrix

movies_features = csr_matrix(movies_matrix.values)

In [27]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(movies_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)